In [174]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [175]:
bls_work_stoppages_file_location = 'Dataset/work_stoppages_BLS.csv'  

def read_numeric_cell_with_commas(cell):
    try:
        return int(cell.replace(',',''))
    except:
        return 0

split_string_by_comma = lambda s: pd.Series(str(s).split(','))

dtypes = {
    'Organizations involved': 'category',
    'Ownership': 'category',
}

renamed_columns = {
    'Days idle, cumulative for this work stoppage[3]': 'Cumulative days idle',
    'Number of workers[2]': 'Number of workers',
    'Work stoppage beginning date': 'Beginning date',
    'Work stoppage ending date': 'Ending date',
    'Industry code[1]': 'Industry code'

}

column_converters = {
    'States': split_string_by_comma,
    'Days idle, cumulative for this work stoppage[3]': read_numeric_cell_with_commas, 
    'Number of workers[2]': read_numeric_cell_with_commas
}

other_columns_to_use = ['States', 'Union acronym']


bls_dataframe = pd.read_csv(bls_work_stoppages_file_location, 
    dtype=dtypes, 
    usecols=list(dtypes) + list(renamed_columns) + other_columns_to_use, 
    parse_dates=['Work stoppage beginning date','Work stoppage ending date'],
    converters=column_converters)

bls_dataframe = bls_dataframe.rename(columns=renamed_columns)

bls_dataframe = bls_dataframe.explode(column='States')
bls_dataframe.head()



,Organizations involved,States,Ownership,Industry code,Union acronym,Beginning date,Ending date,Number of workers,Cumulative days idle
0,Marine Towing and Transportation Employers' As...,NY,Private industry,488330,ILA,1988-02-16,1993-12-20,2500,2879500
1,Boeing Company,WA,Private industry,336411,SPEEA,1993-01-19,1993-01-19,21000,21000
2,Boston Gas Company,MA,Private industry,22121,USW,1993-01-24,1993-05-20,1000,83000
3,Bituminous Coal Operators Association,IN,Private industry,212112,UMWA,1993-02-02,1993-03-02,6700,103400
3,Bituminous Coal Operators Association,IL,Private industry,212112,UMWA,1993-02-02,1993-03-02,6700,103400


In [176]:
bls_industry_codes_file_location = 'Dataset/bls_naics_2022_titles_descriptions.csv'

industry_codes_df = pd.read_csv(bls_industry_codes_file_location, usecols=['2022 NAICS','2022 NAICS Short Title']).rename({'2022 NAICS': 'Industry code', '2022 NAICS Short Title': 'Industry Name'})
industry_codes_df.head()

,2022 NAICS,2022 NAICS Short Title
0,11,"Agriculture, Forestry, Fishing and Hunting"
1,111,Crop Production
2,1111,Oilseed and Grain Farming
3,11111,Soybean Farming
4,111110,Soybean Farming


In [177]:

state_abbr = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']  

grouped_by_state = bls_dataframe.groupby(['States']).agg({
    'Number of workers': ['sum', 'mean'],
    'Cumulative days idle': ['sum','mean']
})

grouped_by_state.head(10)

Number of workers               Cumulative days idle              
                     sum          mean                  sum          mean
States                                                                   
                   36500  36500.000000              1204500  1.204500e+06
 AL                 2700   1350.000000                90800  4.540000e+04
 AZ                 4300   4300.000000                12900  1.290000e+04
 CA               164800  23542.857143             17684200  2.526314e+06
 CO               168100  33620.000000             21429500  4.285900e+06
 CT                45200  11300.000000               116700  2.917500e+04
 DC               241200  60300.000000             18366200  4.591550e+06
 DE                89300  29766.666667               982500  3.275000e+05
 FL               169600  24228.571429             17562200  2.508886e+06
 GA               176700  22087.500000             17775400  2.221925e+06

In [178]:
grouped_by_ownership = bls_dataframe.groupby(['Ownership']).agg({
    'Number of workers': ['sum', 'mean'],
    'Cumulative days idle': ['sum','mean']
})

grouped_by_ownership.head()

C:\Users\emmaf\AppData\Local\Temp\ipykernel_10452\3513845020.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_by_ownership = bls_dataframe.groupby(['Ownership']).agg({


Number of workers                \
                                         sum          mean   
Ownership                                                    
Local government                      637200   5845.871560   
Private industry                     8564400  12394.211288   
State and local government             15800  15800.000000   
State government                      851000  17020.000000   

                           Cumulative days idle                 
                                            sum           mean  
Ownership                                                       
Local government                        3512200   32222.018349  
Private industry                      526363000  761740.955137  
State and local government               136500  136500.000000  
State government                        3031100   60622.000000